In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import jieba
import tensorflow as tf


/home/light/App/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('../data/train.csv')

In [3]:
df.groupby('subject').describe()

sentiment_value                                             
                  count      mean       std  min  25%  50%  75%  max
subject                                                             
价格               1273.0 -0.024352  0.450581 -1.0  0.0  0.0  0.0  1.0
内饰                536.0 -0.065299  0.700753 -1.0 -1.0  0.0  0.0  1.0
动力               2732.0  0.002196  0.528218 -1.0  0.0  0.0  0.0  1.0
外观                489.0  0.008180  0.680476 -1.0  0.0  0.0  0.0  1.0
安全性               573.0  0.012216  0.580744 -1.0  0.0  0.0  0.0  1.0
操控               1036.0  0.175676  0.620135 -1.0  0.0  0.0  1.0  1.0
油耗               1082.0  0.012015  0.516914 -1.0  0.0  0.0  0.0  1.0
空间                442.0  0.196833  0.679929 -1.0  0.0  0.0  1.0  1.0
舒适性               931.0 -0.155747  0.608556 -1.0 -1.0  0.0  0.0  1.0
配置                853.0 -0.039859  0.565690 -1.0  0.0  0.0  0.0  1.0

In [4]:
df.head(2)

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高


In [5]:
oh_df = pd.get_dummies(df, columns=['subject'])

In [6]:
oh_df.head(2)

,content_id,content,sentiment_value,sentiment_word,subject_价格,subject_内饰,subject_动力,subject_外观,subject_安全性,subject_操控,subject_油耗,subject_空间,subject_舒适性,subject_配置
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,0,影响,1,0,0,0,0,0,0,0,0,0
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,-1,高,1,0,0,0,0,0,0,0,0,0


In [7]:
theme = '价格'

In [8]:
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stops = set([v.strip() for v in f.readlines()])
    return stops
stops = load_stopwords('../data/stopwords.txt')
def seg(text):
    result = [word for word in jieba.cut(text) if word not in stops]
    return result

In [9]:
def count_words(count_dict, serial):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in serial:
        for word in seg(sentence):
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [10]:
word_counts = {}
count_words(word_counts, oh_df['content'])
            
print("Size of Vocabulary:", len(word_counts))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.819 seconds.
Prefix dict has been built succesfully.


Size of Vocabulary: 16280


In [11]:
def load_embeddings(embedding_file, header=True):
    # Load embeddings
    embeddings_index = {}
    with open(embedding_file) as f:
        if header:
            print(f.readline())
        for line in f:
            line = line.strip()
            if not line:
                break
            try:
                word, vec = line.split(' ', 1)
            except:
                print(line)
            vec = [float(v) for v in vec.split(' ')]
            embedding = np.asarray(vec, dtype='float32')
            embeddings_index[word] = embedding

    print('Word embeddings:', len(embeddings_index))
    return embeddings_index


In [12]:
def save_embedding(word_embedding_index, path):
    first = True
    with open(path, 'w') as f:
        for k, rep in word_embedding_index.items():
            line = '{} {}'.format(k, ' '.join([str(v) for v in rep]))
            if first:
                first = False
            else:
                f.write('\n')
            f.write(line)
        

In [13]:
embeddings_index = load_embeddings('../data/cliped_vec.vec', header=False)

Word embeddings: 12666


In [14]:
def vocab2int2vocab(word_counts, embeddings_index):
    # Limit the vocab that we will use to words that appear ≥ threshold or are in embedding

    #dictionary to convert words to integers
    vocab_to_int = {} 

    value = 0
    for word, count in word_counts.items():
        if word in embeddings_index:
            vocab_to_int[word] = value
            value += 1

    # Special tokens that will be added to our vocab
    codes = ["<UNK>","<PAD>"]   

    # Add codes to vocab
    for code in codes:
        vocab_to_int[code] = len(vocab_to_int)

    # Dictionary to convert integers to words
    int_to_vocab = {}
    for word, value in vocab_to_int.items():
        int_to_vocab[value] = word

    usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

    print("Total number of unique words:", len(word_counts))
    print("Number of words we will use:", len(vocab_to_int))
    print("Percent of words we will use: {}%".format(usage_ratio))
    return vocab_to_int, int_to_vocab

In [15]:
vocab2int, int2vocab = vocab2int2vocab(word_counts, embeddings_index)

Total number of unique words: 16280
Number of words we will use: 11676
Percent of words we will use: 71.72%


In [16]:
def get_embedding_matrix(vocab_to_int, embeddings_index):
    # Need to use 300 for embedding dimensions to match CN's vectors.
    embedding_dim = 300
    nb_words = len(vocab_to_int)
    not_in = 0
    # Create matrix with default values of zero
    word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
    for word, i in vocab_to_int.items():
        if word in embeddings_index:
            word_embedding_matrix[i] = embeddings_index[word]
        elif word == '<PAD>':
            not_in += 1
            new_embedding = np.zeros((embedding_dim, ))
            embeddings_index[word] = new_embedding
            word_embedding_matrix[i] = new_embedding
        else:
            not_in += 1
            # If word not in CN, create a random embedding for it
            new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
            embeddings_index[word] = new_embedding
            word_embedding_matrix[i] = new_embedding

    # Check if value matches len(vocab_to_int)
    print(len(word_embedding_matrix))
    print('not in:', not_in)
    return word_embedding_matrix

In [17]:
embedding_matrix = get_embedding_matrix(vocab2int, embeddings_index)
save_embedding(embeddings_index, '../data/filled.es.vec')

11676
not in: 2


In [18]:
def convert_to_ints(serial, vocab_to_int, word_count, unk_count):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in serial:
        sentence_ints = []
        for word in seg(sentence):
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [19]:
def get_labels(df, theme):
    theme_col = 'subject_{}'.format(theme)
    for index, row in df.iterrows():
        # value = [0, 0, 0, 0]
        # print(row)
        if row[theme_col] == 0:
            key = 0
        else:
            key = int(row['sentiment_value'] + 2)
        yield key
        # value[key] = 1
        # yield value

In [20]:
word_count = 0
unk_count = 0

int_es, word_count, unk_count = convert_to_ints(oh_df['content'], vocab2int, word_count, unk_count)
label = list(get_labels(oh_df, theme))

In [21]:
def pad_sentence_batch(sentence_batch, vocab_to_int, max_sentence=80):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    return [sentence[:max_sentence] + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence[:max_sentence])) for sentence in sentence_batch]

In [22]:
def split_train_val(X, Y, val_size=3000):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    val_idxs = set(indices[:val_size])
    val_X, val_Y = [], []
    train_X, train_Y = [], []
    match_count = 0
    for idx in indices:
        if idx in val_idxs:
            val_X.append(X[idx])
            val_Y.append(Y[idx])
        else:
            train_X.append(X[idx])
            train_Y.append(Y[idx])
    return (train_X, train_Y), (val_X, val_Y)

In [23]:
def get_batch(X, Y=None, num_epochs=4, batch_size=10, echo=True, shuffle=True):
    
    batch_x = []
    if Y:
        batch_y = []
    indices = np.arange(len(X))
    for epoch in range(num_epochs):
        if echo:
            print('-----------epoch:', epoch)
        if shuffle:
            np.random.shuffle(indices)
        for idx in indices:
            x = X[idx]
            if Y:
                y = Y[idx]
            batch_x.append(x)
            if Y:
                batch_y.append(y)
            if len(batch_x) == batch_size:
                bx = pad_sentence_batch(batch_x, vocab2int)
                if Y:
                    yield bx, batch_y
                else:
                    yield bx
                batch_x = []
                if Y:
                    batch_y = []
        

In [24]:
from textcnn import TextCNN
from sklearn.metrics import f1_score

In [25]:
(train_X, train_Y), (val_X, val_Y) = split_train_val(int_es, label, val_size=300)

In [26]:

len(label)

9947

In [27]:
def get_f1(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    return f1

In [28]:
checkpoint = './jiage.ckpt'
work_dir = 'jiage_graph'
test_size = 1000
test_loss = 10000

stop_early = 0
stop = 10
best_f1 = 0

print_step = 5
update_step = 20

with tf.Graph().as_default() as g:
    cnn = TextCNN(seq_len=80, embedding=embedding_matrix, filter_sizes=[2, 3, 4], num_filters=256, num_classes=4, l2_reg=0.01)
    cnn.create_placeholder()
    cnn.create_variable()
    cnn.create_model()
    cnn.create_loss(lr=0.00001)
    with tf.Session(graph=g) as sess:
        writer = tf.summary.FileWriter(work_dir, sess.graph)
        
        # global_step = tf.Variable(0, name='global_step', trainable=False)
        sess.run(tf.global_variables_initializer())

        step = 0
        for batch_x, batch_y in get_batch(train_X, train_Y, num_epochs=6, batch_size=30):
            feed_dict = {
                cnn.x: batch_x,
                cnn.y: batch_y,
                cnn.keep_prob: 0.6,
            }
            # loss_summary = tf.summary.scalar("loss", cnn.loss)
            # loss_smry, loss, _ = sess.run([loss_summary, cnn.loss, cnn.train_op], feed_dict=feed_dict)
            loss, _ = sess.run([cnn.loss, cnn.train_op], feed_dict=feed_dict)
            step += 1
            
            # writer.add_summary(loss_smry, step)
           
            
            if step % print_step == 0:
                print('step:', step, ', loss:', loss)
            if step % update_step == 0:
                val_loss = []
                for batch_x, batch_y in get_batch(val_X, val_Y, num_epochs=1, batch_size=20, echo=False):
                    feed_dict = {
                        cnn.x: batch_x,
                        cnn.y: batch_y,
                        cnn.keep_prob: 1,
                    }
                    y_pred, loss, _ = sess.run([cnn.predictions, cnn.loss, cnn.train_op], feed_dict=feed_dict)
                    val_loss.append(loss)
                update_loss = sum(val_loss)/len(val_loss)
                f1 = get_f1(batch_y, y_pred)
                print('test loss:', update_loss, ', f1:', f1)
                if (update_loss <= test_loss) and (f1 > best_f1):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)
                    test_loss = update_loss
                    if f1 < 0.9:
                        best_f1 = f1
                else:
                    print("No Improvement.")
                    stop_early += 1
                if best_f1 > 0.5:
                    if stop_early == stop:
                        break
                else:
                    stop_early = 0
                   
                update_loss = 0

                


Tensor("x:0", shape=(?, 80), dtype=int32) Tensor("y:0", shape=(?,), dtype=int32) Tensor("keep_prob:0", dtype=float32)
Tensor("concat:0", shape=(?, 1, 1, 768), dtype=float32)
Tensor("dropout/dropout/mul:0", shape=(?, 768), dtype=float32)
Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

-----------epoch: 0
step: 5 , loss: 1.6202083
step: 10 , loss: 1.6048535
step: 15 , loss: 1.626726
step: 20 , loss: 1.6361796


/home/light/App/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


test loss: 1.5847752332687377 , f1: 0.045454545454545456
New Record!
step: 25 , loss: 1.6835632
step: 30 , loss: 1.6409535
step: 35 , loss: 1.6053871
step: 40 , loss: 1.5560058
test loss: 1.5623201131820679 , f1: 0.125
New Record!
step: 45 , loss: 1.4760184
step: 50 , loss: 1.6024399
step: 55 , loss: 1.5318824
step: 60 , loss: 1.6122329


/home/light/App/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


test loss: 1.5407095511754354 , f1: 0.08125
No Improvement.
step: 65 , loss: 1.5756677
step: 70 , loss: 1.519667
step: 75 , loss: 1.5455898
step: 80 , loss: 1.5852976
test loss: 1.5193959871927898 , f1: 0.0
No Improvement.
step: 85 , loss: 1.5692003
step: 90 , loss: 1.4900525
step: 95 , loss: 1.543061
step: 100 , loss: 1.4951389
test loss: 1.498287590344747 , f1: 0.04761904761904762
No Improvement.
step: 105 , loss: 1.4638535
step: 110 , loss: 1.4752098
step: 115 , loss: 1.460563
step: 120 , loss: 1.5317277
test loss: 1.4779767910639445 , f1: 0.024999999999999998
No Improvement.
step: 125 , loss: 1.5072783
step: 130 , loss: 1.525418
step: 135 , loss: 1.4753922
step: 140 , loss: 1.4518906
test loss: 1.4580783605575562 , f1: 0.07894736842105263
No Improvement.
step: 145 , loss: 1.456187
step: 150 , loss: 1.431739
step: 155 , loss: 1.5157124
step: 160 , loss: 1.5400963
test loss: 1.4378921031951903 , f1: 0.10714285714285715
No Improvement.
step: 165 , loss: 1.4614568
step: 170 , loss: 1.4

test loss: 0.9229409257570903 , f1: 0.22222222222222224
No Improvement.
-----------epoch: 3
step: 965 , loss: 0.8675812
step: 970 , loss: 0.97074866
step: 975 , loss: 0.85505617
step: 980 , loss: 0.83300865
test loss: 0.9154188156127929 , f1: 0.3063063063063063
No Improvement.
step: 985 , loss: 0.8676422
step: 990 , loss: 0.8576185
step: 995 , loss: 0.9666559
step: 1000 , loss: 0.9759339
test loss: 0.9079889933268229 , f1: 0.29629629629629634
No Improvement.
step: 1005 , loss: 1.0154312
step: 1010 , loss: 0.8812673
step: 1015 , loss: 0.9460286
step: 1020 , loss: 0.80418277
test loss: 0.9008304953575135 , f1: 0.3063063063063063
No Improvement.
step: 1025 , loss: 0.8684019
step: 1030 , loss: 0.93431205
step: 1035 , loss: 0.91892844
step: 1040 , loss: 0.85865766
test loss: 0.8939104398091634 , f1: 0.31578947368421056
No Improvement.
step: 1045 , loss: 0.88534063
step: 1050 , loss: 0.77127135
step: 1055 , loss: 0.82411844
step: 1060 , loss: 0.88456786
test loss: 0.8871761480967204 , f1: 0.

step: 1825 , loss: 0.8707172
step: 1830 , loss: 0.70080113
step: 1835 , loss: 0.50136185
step: 1840 , loss: 0.65137464
test loss: 0.7191496372222901 , f1: 0.48717948717948717
No Improvement.
step: 1845 , loss: 0.5800684
step: 1850 , loss: 0.6437214
step: 1855 , loss: 0.70143056
step: 1860 , loss: 0.7048693
test loss: 0.7164453486601512 , f1: 1.0
New Record!
step: 1865 , loss: 0.79211366
step: 1870 , loss: 0.61585444
step: 1875 , loss: 0.7532644
step: 1880 , loss: 0.72311187
test loss: 0.7139386455217998 , f1: 0.22222222222222224
No Improvement.
step: 1885 , loss: 0.56182885
step: 1890 , loss: 0.5821235
step: 1895 , loss: 0.61996716
step: 1900 , loss: 0.9036221
test loss: 0.711421529452006 , f1: 0.2857142857142857
No Improvement.
step: 1905 , loss: 0.521705
step: 1910 , loss: 0.5515935
step: 1915 , loss: 0.6188651
step: 1920 , loss: 0.6932901
test loss: 0.7090383172035217 , f1: 0.48717948717948717
No Improvement.
step: 1925 , loss: 0.5619769


### 预测

In [29]:
test = pd.read_csv('../data/test_public.csv')
test.shape

(2364, 2)

In [30]:
word_count = 0
unk_count = 0

test_X, word_count, unk_count = convert_to_ints(test['content'], vocab2int, word_count, unk_count)

In [31]:

loaded_graph = tf.Graph()
final_result = None
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    x = loaded_graph.get_tensor_by_name('x:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    output = loaded_graph.get_tensor_by_name('output/predictions:0')
    #Multiply by batch_size to match the model's input parameters
    for t_x in get_batch(test_X, num_epochs=1, batch_size=591, echo=False, shuffle=False):
        result = sess.run(output, {x: t_x, keep_prob: 1.0})
        if final_result is None:
            final_result = result
        else:
            final_result = np.concatenate((final_result, result), axis=0)
print(final_result.shape)

INFO:tensorflow:Restoring parameters from ./jiage.ckpt
(2364,)


In [32]:
test['pred'] = final_result
test.head(20)

,content_id,content,pred
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！,0
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊,0
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜,0
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。,0
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪,0
5,cudSKGTyMLbIt8k3,2.0时尚优惠两万现金吗？还有其他赠品吗？,0
6,U5RMQtopwFzDVxHm,27.5 相比较优惠的少,0
7,Kh0ayJjuTrzwPAs1,综合优惠两万，不是现金优惠两万，送的垃圾东西都包含在内.大忽悠.,0
8,YZovRtEAyQJgTi1I,差不多15000左右的优惠,0
9,BeSIAUzoTWH9muYl,恭喜恭喜，这个配置性价比很高,0


In [33]:
test.describe()
test.to_csv('../output/jiage.csv')